**Name:** Alex Medina

**File:** Comprehensive Test

## **SECTION 0:** Setup

In [1]:
import os
os.environ['picaso_refdata'] = r'C:\Users\Alex\Desktop\Picaso\picaso\reference' #THIS MUST GO BEFORE YOUR IMPORT STATEMENT
os.environ['PYSYN_CDBS'] = r'C:\Users\Alex\Desktop\Picaso\grp\redcat\trds' #this is for the stellar data discussed below.

#General
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
import sys
import os
import json
from pathlib import Path

# Picaso
from picaso import justdoit as jdi
from picaso import justplotit as jpi

# Virga
from virga import justdoit as vj
from virga import justplotit as cldplt

# Other
from bokeh.models import Legend
from bokeh.palettes import Category10
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## **SECTION 1:** Helper methods

In [2]:
# Selecting clouds instead of harcoding cloud species
# Select using virga condensation

def autoselect_clouds(
    P_bar, T_K, mmw,
    p_cutoff=1e-2,
    dT_min=5.0,                 # [K] supersaturation margin: Tcond - T >= dT_min somewhere
    dlogP_min=0.2,              # require >= 0.2 dex of condensing column above base
    require_source_below=True,  # simple “source” sanity check
    p_source_min=0.1,           # require base >= this pressure (deeper than e.g. 0.1 bar)
    species=None,               # override list if desired
    return_details=True,
    # --- optional “loftability” screen (very crude; off by default) ---
    check_loftability=False,
    Kzz_cms2=None, g_cgs=None, r_eff_um=None, rho_p_gcc=None,
    loftability_min=0.3,        # require Kzz/(v_set*H) >= this
):
    """
    Choose condensates that plausibly form a cloud deck above ~p_cutoff
    with minimal physical guardrails.

    Parameters
    ----------
    P_bar : array
        Pressure profile in bar (decreasing upward is fine).
    T_K : array
        Temperature profile in K.
    mmw : float or array
        Mean molecular weight used by virga condensation curves.
    p_cutoff : float
        Only consider levels with P >= p_cutoff (e.g., >= 10 mbar).
    dT_min : float
        Minimum margin (Tcond - T) in K at/above the base to avoid grazing intersections.
    dlogP_min : float
        Minimum vertical extent (in log10 P) of condensing column above base.
    require_source_below : bool
        If True, require the cloud base to be at or deeper than p_source_min (a crude “source” check).
    p_source_min : float
        Minimum acceptable base pressure (bar) if require_source_below is True.
    species : list[str]
        Optional species list. Defaults to common condensates for BDs/EGPs.
    return_details : bool
        If True, return (selected_species, details_dict). Else return just the list.
    check_loftability : bool
        If True, apply a crude loftability/rainout screen using Kzz, g, r_eff, rho_p.
    Kzz_cms2 : float
        Eddy diffusion coefficient [cm^2/s] for loftability check.
    g_cgs : float
        Gravity [cm/s^2] for loftability check.
    r_eff_um : float
        Effective particle radius [micron] for loftability check.
    rho_p_gcc : float
        Particle material density [g/cm^3] for loftability check.
    loftability_min : float
        Minimum Kzz/(v_set*H) value to accept cloud (0.3–1 is a typical range).

    Returns
    -------
    sel : list of species strings
    details : dict (if return_details)
        Per-species dict with fields:
        - base_P_bar
        - has_margin
        - logP_span
        - passes_source
        - passes_loftability
        - cond_mask (bool array over pressure grid considered)
    """

    cloud_species = species or [
        "Al2O3","Cr","Fe","H2O","KCl","Mg2SiO4",
        "MgSiO3","MnS","NH3","Na2S","ZnS"
    ]

    P = np.asarray(P_bar, float)
    T = np.asarray(T_K,   float)
    if P.shape != T.shape:
        raise ValueError("P_bar and T_K must have the same shape")

    # Work on the portion deeper than the cutoff
    valid = np.isfinite(P) & np.isfinite(T) & (P >= float(p_cutoff))
    if not np.any(valid):
        return ([] if not return_details else ([], {}))

    # Ensure monotonicity (for base-finding) by sorting by pressure descending
    order = np.argsort(P[valid])[::-1]  # deep -> high
    Pv = P[valid][order]
    Tv = T[valid][order]

    sel = []
    details = {}

    # Pre-compute scale height for loftability if requested (isothermal-ish approx).
    # H = k_B T / (mu m_H g)
    # Use a representative T (~ median of Tv) and mmw
    if check_loftability:
        if None in (Kzz_cms2, g_cgs, r_eff_um, rho_p_gcc):
            raise ValueError("Loftability check requested but Kzz_cms2, g_cgs, r_eff_um, rho_p_gcc not all provided.")
        k_B = 1.380649e-16      # erg/K
        m_H = 1.6735575e-24     # g
        mu  = float(np.nanmedian(np.atleast_1d(mmw)))
        Tchar = float(np.nanmedian(Tv))
        H_cm = (k_B * Tchar) / (mu * m_H * g_cgs)  # [cm]

    for sp in cloud_species:
        # Condensation temperature along the valid segment
        _, Tcond_all = vj.condensation_t(sp, 1.0, mmw, pressure=Pv)  # Virga expects bar, same as Pv
        Tcond = np.asarray(Tcond_all, float)

        good = np.isfinite(Tcond) & np.isfinite(Tv)
        if not np.any(good):
            details[sp] = dict(
                base_P_bar=None, has_margin=False, logP_span=0.0,
                passes_source=False, passes_loftability=True, cond_mask=np.zeros_like(Pv, bool)
            )
            continue

        # Where it could condense
        cond_mask = good & (Tv <= Tcond)  # supersaturated or at saturation

        if not np.any(cond_mask):
            details[sp] = dict(
                base_P_bar=None, has_margin=False, logP_span=0.0,
                passes_source=False, passes_loftability=True, cond_mask=cond_mask
            )
            continue

        # Cloud base = deepest pressure where cond_mask is True
        base_idx = np.argmax(cond_mask)  # first True along deep->high if any True earlier? No; need deepest True
        # np.argmax returns first index of max; but cond_mask is bool. Because array starts deep->high,
        # we actually want the first True encountered from the deep side. If the first element is False
        # until the first True, np.argmax gives that first True. If there are leading Trues, it's okay.
        # To be robust, do:
        where_true = np.where(cond_mask)[0]
        base_idx   = int(where_true[0])
        base_P     = float(Pv[base_idx])

        # Vertical extent above base where it stays condensing
        # Note: array is deep->high, so "above base" means indices >= base_idx
        above_mask = np.zeros_like(cond_mask, bool)
        above_mask[base_idx:] = cond_mask[base_idx:]
        if np.any(above_mask):
            P_above = Pv[above_mask]
            # extent in log10P
            logP_span = np.log10(P_above.max()) - np.log10(P_above.min()) if P_above.size > 1 else 0.0
        else:
            logP_span = 0.0

        # Supersaturation margin check (somewhere above or at base)
        has_margin = np.any((Tcond - Tv >= float(dT_min)) & above_mask)

        # Source sanity: require base not too high up (optional)
        passes_source = (base_P >= float(p_source_min)) if require_source_below else True

        # Optional crude loftability
        passes_loft = True
        if check_loftability:
            # Stokes terminal velocity (very crude; Cunningham slip etc. ignored)
            # v_set ~ 2/9 * (Δρ/η) * g * r^2  — but we’ll use a microphysics-inspired scaling
            # Here, use Ackerman & Marley-like f_sed proxy via Kzz/(v_set*H).
            # Without knowing viscosity, use an empirical v_set ~ g*r/(C) scaling.
            # We only want a rejector for obviously un-loftable big/denser grains.
            r_cm = r_eff_um * 1e-4    # μm -> cm
            # crude scaling constant C chosen so that r~1–10 μm is typically loftable for Kzz~1e8–1e10
            C = 50.0
            v_set = (g_cgs * r_cm) / C  # [cm/s], extremely rough
            ratio = Kzz_cms2 / (max(v_set, 1e-20) * H_cm)
            passes_loft = (ratio >= float(loftability_min))

        ok = (logP_span >= float(dlogP_min)) and has_margin and passes_source and passes_loft

        details[sp] = dict(
            base_P_bar=base_P, has_margin=bool(has_margin), logP_span=float(logP_span),
            passes_source=bool(passes_source), passes_loftability=bool(passes_loft),
            cond_mask=cond_mask
        )
        if ok:
            sel.append(sp)

    return (sel, details) if return_details else sel


In [3]:
# Cloud selection depends on MMW, but I feel that I am making loops here
def compute_mmw(profile_dict, fallback=2.36):
    """
    Computing mmw from Sonora profile if present; else use solar mmw.
    """
    mu = profile_dict.get("mu") or profile_dict.get("MU")
    if mu is None:
        return float(fallback)
    mu = np.asarray(mu, float)
    if mu.ndim == 0:
        return float(mu)
    return float(np.mean(mu))

In [4]:
# To have a specific filename structure
def make_case_filename(Teff, gravity, kzz, fsed, output_dir, ext=".npz"):
    # Whether cloud or clear case
    cloud_tag = "clouds"
    base = f"T{int(round(Teff))}G{int(round(gravity))}{cloud_tag}"

    base += f"_Kzz{float(kzz):.2e}"
    base += f"_fsed{float(fsed):.2f}"

    return os.path.join(output_dir, base + ext)

# Save all files in one place
def make_master_filename(n_spectra, waverange, R, output_dir, clouds=True, ext=".npz"):
    cloud_tag = "clouds" if clouds else "clear"
    # Encode basic config in the filename
    wmin, wmax = waverange
    base = f"N{n_spectra}_R{int(R)}_{wmin:.2f}-{wmax:.2f}um_{cloud_tag}"
    return os.path.join(output_dir, base + ext)

## **SECTION 2:** Main function

In [5]:
def bd_spectrum(waverange, gravity, Teff, kzz, fsed, mh, R,
                opacity_db, sonora_db, virga_db):
    """
    Compute a BD emission spectrum with optional Virga clouds.
    """

    # Opacity & inputs
    opa = jdi.opannection(wave_range=list(waverange), filename_db=opacity_db)
    bd = jdi.inputs(calculation="browndwarf")

    # Basic inputs
    bd.phase_angle(0)
    bd.gravity(gravity, gravity_unit=u.Unit('m/s**2'))
    bd.sonora(sonora_db, Teff)

    # Inject Kzz (must match pressure grid length)
    prof = bd.inputs['atmosphere']['profile']
    P = np.asarray(prof["pressure"], float)
    T = np.asarray(prof["temperature"], float)
    mmw = compute_mmw(prof)
    
    bd.inputs["atmosphere"]["profile"]["kz"] = [float(kzz)] * len(P)
    
    # Clouds
    cloud_list, _ = autoselect_clouds(
    P_bar=P, T_K=T, mmw=mmw,
    p_cutoff=1e-2,     # 10 mbar
    dT_min=5.0,        # need at least 5 K of margin
    dlogP_min=0.2,     # ≥ ~0.2 dex vertical span
    require_source_below=True,
    p_source_min=0.1,  # base must be ≥ 0.1 bar
    # Loftability screen if you want it:
    check_loftability=False, # switch to True to enable
    Kzz_cms2=1e9, g_cgs=1e5, r_eff_um=3.0, rho_p_gcc=3.0)

    bd.virga(cloud_list, virga_db, fsed=float(fsed), mh=mh, mmw=mmw)

    out = bd.spectrum(opa, full_output=True)

    # Convert to F_nu, then regrid to constant R in wavenumber
    wn, th = out["wavenumber"], out["thermal"]  # cm^-1 and erg/cm^2/s/cm
    # Convert to F_nu on a wavelength grid first
    wm = 1e4 / wn
    flamy    = th * 1e-8  # per angstrom instead of per cm
    sp = jdi.psyn.ArraySpectrum(wm, flamy, 
                                waveunits='um',
                                fluxunits='FLAM')
    sp.convert('um')
    sp.convert('Fnu')  # erg/cm^2/s/Hz

    wav_um, F_nu = sp.wave, sp.flux # micron and erg/cm^2/s/Hz
    out['fluxnu'] = F_nu

    # Regrid at constant resolving power in WAVENUMBER space
    k_cm1_in = 1e4 / wav_um
    k_cm1_rg, F_nu_rg = jdi.mean_regrid(k_cm1_in, F_nu, R=R)  # returns k in cm^-1


    # Convert back to wavelength (µm) for plotting
    lam_um_rg = 1e4 / k_cm1_rg
    # ensure ascending wavelength for plotting
    order = np.argsort(lam_um_rg)
    lam_um_rg = lam_um_rg[order]
    F_nu_rg   = F_nu_rg[order]

    out['regridx'] = lam_um_rg  # now micron
    out['regridy'] = F_nu_rg
    return lam_um_rg, F_nu_rg, cloud_list

## **SECTION 3:** Configuration

In [6]:
# Directories
input_dir  = r"C:\Users\Alex\Desktop\Picaso\data\sonora" # Sonora db
virga_dir  = r"C:\Users\Alex\Desktop\Picaso\data\virga" # Virga
opacit_dir = None # Opacity db
# WILL CHANGE ONCE RUNNING ON NEWTON AND TP CORRECT
# r"/groups/tkaralidi/opacity_500k_for_R5000_egpoutput.db"
#output_dir = r"C:\Users\Alex\Desktop\Picaso\outputs" # Output store
output_dir = Path(r"C:\Users\Alex\Desktop\Picaso\outputs")

use_clouds = True # Clouds on/off

# Wavelength and resolution
wav_range       = (0.3, 3.0) # microns
res_R           = 300 
# WILL CHANGE ONCE RUNNING ON NEWTON AND TP CORRECT
# res_R = 5000

# Cloud microphysics
MH              = 1.0   # [M/H] metallicity factor ~ solar

# Grid space
Teff_range      = (700, 2000)     # K
gravity_range   = (50.0, 1500.0)  # m/s^2
fsed_range      = (0.5, 5.0)      # dimensionless
Kzz_range       = (1e8, 3e10)     # cm^2/s

# Sampling
seed            = 43
N_spectra       = 5 # Number of spectra to generate for sampling

# Knobs
save_per_case   = True   # Also save per-spectrum files with friendly names
kzz_loguniform  = True   # Log-uniform sampling for Kzz (typical)
VERBOSE         = True

## **SECTION 4:** Execute ONE Model

In [7]:
# Since just ONE model, we need just ONE parameter set
Teff_one      = 1700     # K
gravity_one   = 100      # m/s^2
fsed_one      = 1     # dimensionless

if kzz_loguniform:
    kzz_one   = 10.0**(np.log10(1e10)) # cm^2/s
else:
    kzz_one   = 1e10                   # cm^2/s

# Print parameters
if VERBOSE:
    print("Chosen single-model parameters:")
    print(f"Teff  = {Teff_one:.1f} K")
    print(f"g     = {gravity_one:.1f} m/s^2")
    print(f"f_sed = {fsed_one:.2f}")
    print(f"Kzz   = {kzz_one:.3e} cm^2/s")
    print(f"[M/H] = {MH:.2f}")
    print(f"R     = {res_R:d}")
    print(f"Waverange (micron): {wav_range[0]}–{wav_range[1]}")

# Run ONE model (bd_spectrum returns wavenumber [cm^-1] and Fnu regridded at constant R in wavenumber)
wm, Fnu, _ = bd_spectrum(waverange=wav_range,
                      gravity=gravity_one,
                      Teff=Teff_one,
                      kzz=kzz_one,
                      fsed=fsed_one,
                      mh=MH,
                      R=res_R,
                      opacity_db=opacit_dir,
                      sonora_db=input_dir,
                      virga_db=virga_dir)

# For plotting
one_model_meta = {"Teff_K": Teff_one, "g_mps2": gravity_one, "fsed": fsed_one, "Kzz_cm2s": kzz_one,
                  "MH": MH, "R": res_R, "wav_um_min": float(wm.min()), "wav_um_max": float(wm.max())}

Chosen single-model parameters:
Teff  = 1700.0 K
g     = 100.0 m/s^2
f_sed = 1.00
Kzz   = 1.000e+10 cm^2/s
[M/H] = 1.00
R     = 300
Waverange (micron): 0.3–3.0


In [8]:
# Single plot

src = ColumnDataSource(data=dict(lam_um=wm, Fnu=Fnu))

title_txt = (f"BD Spectrum — Teff={one_model_meta['Teff_K']:.0f} K, "
             f"g={one_model_meta['g_mps2']:.0f} m s⁻², f_sed={one_model_meta['fsed']:.2f}, "
             f"Kzz={one_model_meta['Kzz_cm2s']:.1e} cm² s⁻¹, [M/H]={one_model_meta['MH']:.1f}, "
             f"R={one_model_meta['R']}")

p = figure(title=title_txt,
           x_axis_label="Wavelength (micron)",
           y_axis_label="Fnu (erg cm^-2 s^-1 Hz^-1)",
           sizing_mode="stretch_width",
           height=420,
           tools="pan,wheel_zoom,box_zoom,reset,save")


# Choose a simple line color from Category10
color = Category10[3][0]
p.line('lam_um', 'Fnu', source=src, line_width=2, color=color, legend_label="Thermal emission")

p.legend.location = "top_right"
p.legend.click_policy = "hide"

show(p)

## **SECTION 5:** Execute MULTIPLE Models

In [9]:
# For multiple models
# Ala Peter, I want to have same T and g grid
# But vary fsed and kzz

# CAN CHANGE FOR WHAT PETER HAS AND LOOP IT BUT FOR NOW FIZED
Teff_fix = 1700.0  # K
g_fix    = 100.0   # m/s^2
fsed_low, fsed_high = fsed_range
kzz_low, kzz_high = Kzz_range

# RNG
rng = np.random.default_rng(seed)

# Sample f_sed and kzz
fsed_samples = rng.uniform(fsed_low, fsed_high, size=N_spectra)

if kzz_loguniform:
    logK_low, logK_high = np.log10(kzz_low), np.log10(kzz_high)
    kzz_samples = 10.0 ** rng.uniform(logK_low, logK_high, size=N_spectra)
else:
    kzz_samples = rng.uniform(kzz_low, kzz_high, size=N_spectra)

# Parameters
if VERBOSE:
    print(f"Sampling N={N_spectra} spectra at Teff={Teff_fix:.0f} K, g={g_fix:.0f} m/s^2")
    print(f"f_sed range U[{fsed_low:.2f}, {fsed_high:.2f}]")
    if kzz_loguniform:
        print(f"Kzz range logU[{logK_low:.2f}, {logK_high:.2f}] (cm^2/s)")
    else:
        print(f"Kzz rangeU[{kzz_low:.1e}, {kzz_high:.1e}] (cm^2/s)")

# Run the spectra
def run_bd_random(fsed, kzz, Teff=Teff_fix, g=g_fix):
    out = bd_spectrum(waverange=wav_range,
                      gravity=g,
                      Teff=Teff,
                      kzz=float(kzz),
                      fsed=float(fsed),
                      mh=MH,
                      R=res_R,
                      opacity_db=opacit_dir,
                      sonora_db=input_dir,
                      virga_db=virga_dir)
    
    # Accept (x, Fnu) or (x, Fnu, ...)
    if not (isinstance(out, tuple) and len(out) >= 2):
        raise RuntimeError("bd_spectrum must return at least (x, Fnu).")
    x, F = out[0], out[1]
    lam_um = np.asarray(x, float)
    Fnu    = np.asarray(F, float)
    idx    = np.argsort(lam_um)
    return lam_um[idx], Fnu[idx]

# Run all N draws
multi_specs = []
for i in range(N_spectra):
    fsed_i, kzz_i = float(fsed_samples[i]), float(kzz_samples[i])
    lam_um, Fnu = run_bd_random(fsed=fsed_i, kzz=kzz_i)
    multi_specs.append(dict(lam_um=lam_um,
                            Fnu=Fnu,
                            fsed=fsed_i,
                            kzz=kzz_i,
                            Teff=float(Teff_fix),
                            g=float(g_fix)))

multi_meta = {"Teff_K": Teff_fix, "g_mps2": g_fix, "R": res_R, "MH": MH,
              "N": N_spectra,
              "lam_range": (float(multi_specs[0]["lam_um"][0]), float(multi_specs[0]["lam_um"][-1]))}

Sampling N=5 spectra at Teff=1700 K, g=100 m/s^2
f_sed range U[0.50, 5.00]
Kzz range logU[8.00, 10.48] (cm^2/s)


In [10]:
# Plotting the spectra for varying fsed and kzz values

title_txt = (f"BD spectra — N={multi_meta['N']}, " 
             f"Teff={multi_meta['Teff_K']:.0f} K, "
             f"g={multi_meta['g_mps2']:.0f} m s^-2; [M/H]={multi_meta['MH']:.1f}, "
             f"R={multi_meta['R']}")

p = figure(title=title_txt,
           x_axis_label="Wavelength (micron)",
           y_axis_label="Fnu (erg cm^-2 s^-1 Hz^-1)",
           sizing_mode="stretch_width",
           height=520,
           tools="pan,wheel_zoom,box_zoom,reset,save")

palette = Category10[max(3, min(10, len(multi_specs)))]
for i, s in enumerate(multi_specs):
    src = ColumnDataSource(dict(lam_um=s["lam_um"], Fnu=s["Fnu"]))
    label = f"f_sed={s['fsed']:.2g}, Kzz={s['kzz']:.1e}"
    p.line('lam_um', 'Fnu', source=src, line_width=2,
           color=palette[i % len(palette)], legend_label=label)

p.legend.location = "top_right"
p.legend.click_policy = "hide"
p.legend.label_text_font_size = "10pt"

show(p)

## **SECTION 6:** Generating and Saving Multple Models in .NPZ

In [11]:
# Build arrays X (params) and Y (spectra), and confirm a common wavelength grid
N = len(multi_specs)
lam_ref = multi_specs[0]["lam_um"].astype(float)
L = lam_ref.size

X = np.zeros((N, 4), dtype=float)   # [Teff, g, f_sed, Kzz]
Y = np.zeros((N, L), dtype=float)

for i, s in enumerate(multi_specs):
    lam_i = s["lam_um"].astype(float)
    Fnu_i = s["Fnu"].astype(float)

    # Safety: if any spectrum somehow differs in wavelength grid, interpolate to lam_ref
    if lam_i.shape != lam_ref.shape or not np.allclose(lam_i, lam_ref, rtol=0, atol=0):
        Fnu_i = np.interp(lam_ref, lam_i, Fnu_i)

    X[i, :] = [s["Teff"], s["g"], s["fsed"], s["kzz"]]
    Y[i, :] = Fnu_i

# ONE master file
master_name = f"T{int(Teff_fix)}_G{int(g_fix)}_MH{float(MH):.2f}_R{int(res_R)}_N{N}.npz"
master_path = output_dir / master_name
np.savez_compressed(master_path, x=X, y=Y, wavelength_um=lam_ref)
print(f"Saved master: {master_path}")

# AND saving per case
if save_per_case:
    for i in range(N):
        Teff_i, g_i, fsed_i, kzz_i = X[i]
        # Short, readable filename
        fname = (f"T{int(Teff_i)}_G{int(g_i)}_fsed{fsed_i:.2f}_Kzz{kzz_i:.2e}.npz")
        fpath = output_dir / fname
        np.savez_compressed(fpath,
                            x=X[i],                 # (4,)
                            y=Y[i],                 # (L,)
                            wavelength_um=lam_ref)  # (L,)
    print(f"Saved {N} per-case files to {output_dir}")

Saved master: C:\Users\Alex\Desktop\Picaso\outputs\T1700_G100_MH1.00_R300_N5.npz
Saved 5 per-case files to C:\Users\Alex\Desktop\Picaso\outputs


## **SECTION 7:** Visualize Generate Spectra

In [12]:
# Now verifying that the information was stored
# And stored correctly

def plot_npz_file(npz_path, idx=None, title=None):
    """
    Plot a single .npz file.
    - Per-case file: ignores idx (since it stores one spectrum).
    - Master file: set idx to choose which row to plot.
    """
    d = np.load(npz_path)
    lam = d["wavelength_um"].astype(float)

    # Master vs per-case
    if d["y"].ndim == 2:
        if idx is None:
            idx = 0
        F = d["y"][idx].astype(float)
        x_row = d["x"][idx].astype(float)
    else:
        F = d["y"].astype(float)
        x_row = d["x"].astype(float)

    if title is None:
        Teff, g, fsed, kzz = x_row
        title = f"{Path(npz_path).name} — Teff={Teff:.0f}, g={g:.0f}, f_sed={fsed:.2f}, Kzz={kzz:.2e}"

    p = figure(title=title, x_axis_label="Wavelength (micron)",
               y_axis_label="Fnu (erg cm^-2 s^-1 Hz^-1)",
               sizing_mode="stretch_width", height=420,
               tools="pan,wheel_zoom,box_zoom,reset,save")
    src = ColumnDataSource(dict(lam_um=lam, Fnu=F))
    p.line('lam_um', 'Fnu', source=src, line_width=2)
    show(p)

def overlay_npz_from_folder(folder, max_plots=5):
    """
    Overlay up to max_plots spectra from per-case files OR randomly sample from master files.
    """
    folder = Path(folder)
    paths = sorted(folder.glob("*.npz"))
    if not paths:
        print("No .npz files found.")
        return

    p = figure(title=f"Overlay check — up to {max_plots} spectra",
               x_axis_label="Wavelength (micron)",
               y_axis_label="Fnu (erg cm^-2 s^-1 Hz^-1)",
               sizing_mode="stretch_width", height=520,
               tools="pan,wheel_zoom,box_zoom,reset,save")

    palette = Category10[max(3, min(10, max_plots))]
    plotted = 0

    for path in paths:
        if plotted >= max_plots:
            break
        d = np.load(path)
        lam = d["wavelength_um"].astype(float)
        y = d["y"]

        # Per-case
        if y.ndim == 1:
            F = y.astype(float)
            p.line(lam, F, line_width=2, color=palette[plotted % len(palette)],
                   legend_label=path.name)
            plotted += 1

        # Master: sample a few rows
        else:
            rows = min(y.shape[0], max_plots - plotted)
            for i in range(rows):
                F = y[i].astype(float)
                p.line(lam, F, line_width=2, color=palette[plotted % len(palette)],
                       legend_label=f"{path.name} [i={i}]")
                plotted += 1
                if plotted >= max_plots:
                    break

    p.legend.location = "top_right"
    p.legend.click_policy = "hide"
    show(p)


In [13]:
# Examples

# Per file
plot_npz_file(r"C:\Users\Alex\Desktop\Picaso\outputs\T1700_G100_fsed4.28_Kzz1.10e+09.npz")

# Plot i=3 from a master file
plot_npz_file(r"C:\Users\Alex\Desktop\Picaso\outputs\T1700_G100_MH1.00_R300_N5.npz", idx=1)

# Overlay a from a folder
overlay_npz_from_folder(r"C:\Users\Alex\Desktop\Picaso\outputs", max_plots=6)

## **SECTION 8:** View Sample Stats

In [14]:
def inspect_npz(npz_path, show_table=True, max_rows=10):
    """
    Print shapes, input ranges, and flux stats for a saved .npz.
    Works for both per-case (x=(4,), y=(L,)) and master (x=(N,4), y=(N,L)).
    """
    npz_path = Path(npz_path)
    d = np.load(npz_path)

    # Normalize to batch form
    lam = d["wavelength_um"].astype(float)
    x_raw = d["x"]
    y_raw = d["y"]

    if x_raw.ndim == 1:   # per-case
        X = x_raw[None, :].astype(float)   # (1,4)
        Y = y_raw[None, :].astype(float)   # (1,L)
    else:                 # master
        X = x_raw.astype(float)            # (N,4)
        Y = y_raw.astype(float)            # (N,L)

    # Shapes
    print(f"X shape: {X.shape}  (columns: Teff[K], g[m/s^2], f_sed[-], Kzz[cm^2/s])")
    print(f"Y shape: {Y.shape}  (flux_nu per spectrum)")
    print(f"λ shape: {lam.shape} (micron)\n")

    # Input ranges
    Teff = X[:, 0]; g = X[:, 1]; fsed = X[:, 2]; Kzz = X[:, 3]
    print("Input ranges:")
    print(f"  Teff [K]    : {np.min(Teff):.1f} to {np.max(Teff):.1f}")
    print(f"  g [m/s^2]   : {np.min(g):.2f} to {np.max(g):.2f}")
    print(f"  f_sed [-]   : {np.min(fsed):.2f} to {np.max(fsed):.2f}")
    print(f"  Kzz [cm^2/s]: {np.min(Kzz):.3e} to {np.max(Kzz):.3e}\n")

    # Flux stats across all spectra
    ymin = np.min(Y)
    ymax = np.max(Y)
    print("Flux stats across all spectra:")
    print(f"  min: {ymin:.6e}  max: {ymax:.6e}\n")

    # Optional small table of inputs
    if show_table:
        df = pd.DataFrame(
            X,
            columns=["Teff_K", "g_mps2", "f_sed", "Kzz_cm2s"],
            index=pd.Index(range(X.shape[0]), name="spectrum_idx"),
        )
        # Show a compact view
        print(df.head(max_rows).to_string())

    return X, Y, lam

In [15]:
# Examples
# Per-case file:
inspect_npz(r"C:\Users\Alex\Desktop\Picaso\outputs\T1700_G100_fsed3.44_Kzz3.60e+08.npz")

# Master file:
#inspect_npz(r"C:\Users\Alex\Desktop\Picaso\outputs\T1700_G100_MH1.00_R300_N5.npz", show_table=True, max_rows=10)

X shape: (1, 4)  (columns: Teff[K], g[m/s^2], f_sed[-], Kzz[cm^2/s])
Y shape: (1, 691)  (flux_nu per spectrum)
λ shape: (691,) (micron)

Input ranges:
  Teff [K]    : 1700.0 to 1700.0
  g [m/s^2]   : 100.00 to 100.00
  f_sed [-]   : 3.44 to 3.44
  Kzz [cm^2/s]: 3.603e+08 to 3.603e+08

Flux stats across all spectra:
  min: 1.839407e-16  max: 9.539187e-07

              Teff_K  g_mps2     f_sed      Kzz_cm2s
spectrum_idx                                        
0             1700.0   100.0  3.435347  3.602650e+08


(array([[1.70000000e+03, 1.00000000e+02, 3.43534668e+00, 3.60265025e+08]]),
 array([[3.16138977e-14, 3.57368320e-14, 3.79194144e-14, 2.19092824e-14,
         4.44020392e-14, 4.98891616e-14, 5.37668724e-14, 5.72102591e-14,
         5.89251768e-14, 3.82668503e-14, 3.05129404e-14, 6.92923131e-14,
         7.65224329e-14, 8.06189495e-14, 8.35115764e-14, 8.52668246e-14,
         8.56042172e-14, 8.40366405e-14, 7.97122581e-14, 7.00516379e-14,
         3.34256181e-14, 2.29798160e-14, 4.12750844e-14, 3.08212282e-14,
         1.90718114e-14, 9.72533387e-15, 4.14809340e-15, 1.78742118e-15,
         1.83940747e-16, 4.70330809e-16, 2.94752097e-15, 7.88493791e-15,
         2.07547910e-14, 4.50055832e-14, 8.21937731e-14, 1.27630608e-13,
         1.76117550e-13, 2.24548488e-13, 2.67249196e-13, 2.91154701e-13,
         2.35341709e-13, 3.07144921e-14, 1.71012465e-13, 4.01615836e-13,
         4.92806231e-13, 5.47019802e-13, 5.89045094e-13, 6.25395785e-13,
         6.58465895e-13, 6.89375853e-13, 7.18749

## **SECTION 9:** Cloud Diagnostics

In [16]:
def bd_spectrum_with_cloud_outputs(waverange, gravity, Teff, kzz, fsed, mh, R,
                                   opacity_db, sonora_db, virga_db):
    """
    Same as bd_spectrum but also returns:
      - out_full: PICASO full_output (for photon attenuation plots)
      - cld_out : Virga cloud solution (for radii/Mie diagnostics)
      - cloud_list: species auto-selected (from your autoselect_clouds)
    Returns: lam_um_rg, F_nu_rg, cloud_list, out_full, cld_out
    """
    # Opacity & inputs
    opa = jdi.opannection(wave_range=list(waverange), filename_db=opacity_db)
    bd  = jdi.inputs(calculation="browndwarf")

    # Basic inputs
    bd.phase_angle(0)
    bd.gravity(gravity, gravity_unit=u.Unit('m/s**2'))
    bd.sonora(sonora_db, Teff)

    # Inject Kzz
    prof = bd.inputs['atmosphere']['profile']
    P = np.asarray(prof["pressure"], float)
    T = np.asarray(prof["temperature"], float)
    mmw = compute_mmw(prof)
    bd.inputs["atmosphere"]["profile"]["kz"] = [float(kzz)] * len(P)

    # Clouds
    cloud_list = autoselect_clouds(P, T, mmw)
    cld_out = bd.virga(cloud_list, virga_db, fsed=float(fsed), mh=mh, mmw=mmw)

    # Spectrum
    out = bd.spectrum(opa, full_output=True)
    wn, th = out["wavenumber"], out["thermal"]  # cm^-1, erg/cm^2/s/cm

    # Convert to Fnu and regrid at constant R in wavenumber
    wm = 1e4 / wn
    flamy = th * 1e-8
    sp = jdi.psyn.ArraySpectrum(wm, flamy, waveunits='um', fluxunits='FLAM')
    sp.convert('um'); sp.convert('Fnu')
    wav_um, F_nu = sp.wave, sp.flux
    k_cm1_in = 1e4 / wav_um
    k_cm1_rg, F_nu_rg = jdi.mean_regrid(k_cm1_in, F_nu, R=R)
    lam_um_rg = 1e4 / k_cm1_rg
    order = np.argsort(lam_um_rg)
    lam_um_rg = lam_um_rg[order]; F_nu_rg = F_nu_rg[order]

    return lam_um_rg, F_nu_rg, cloud_list, out["full_output"], cld_out


In [17]:
from bokeh.layouts import column
from bokeh.models import Div

def batch_cloud_sanity(n_check=3, p_for_radii=1e-1):
    picked = list(range(min(n_check, len(multi_specs))))
    for i in picked:
        s = multi_specs[i]
        Teff_i, g_i, fsed_i, kzz_i = s["Teff"], s["g"], s["fsed"], s["kzz"]
        print(f"\n=== Case {i}: T={Teff_i:.0f} K, g={g_i:.0f} m/s², f_sed={fsed_i:.2f}, Kzz={kzz_i:.2e} cm²/s ===")

        _, _, cloud_list, out_full, cld_out = bd_spectrum_with_cloud_outputs(
            waverange=wav_range,
            gravity=g_i, Teff=Teff_i, kzz=kzz_i, fsed=fsed_i, mh=MH, R=res_R,
            opacity_db=opacit_dir, sonora_db=input_dir, virga_db=virga_dir
        )

        print("Cloud species (auto-selected):", cloud_list)

        # Photon attenuation
        show(jpi.photon_attenuation(out_full, plot_width=550, plot_height=280,
                                    title=f"Photon attenuation (case {i})"))

        # Particle radii at one pressure
        radii_layout, dndr = cldplt.radii(cld_out, at_pressure=float(p_for_radii))
        caption = Div(text=f"<b>Particle radii at P={p_for_radii:g} bar (case {i})</b>")
        show(column(caption, radii_layout))

# --- Example ---
batch_cloud_sanity(n_check=3, p_for_radii=1e-1)



=== Case 0: T=1700 K, g=100 m/s², f_sed=3.44, Kzz=3.60e+08 cm²/s ===


TypeError: unhashable type: 'list'